In [1]:
import inspect
import sys
sys.path.append("/home/administrator/Documenti/APDFT/prototyping/hessian/AAFF/Libs/")
from aaff import aaff,alc_deriv_grad_nuc,alc_differential_grad_nuc
from FcMole import FcM
from pyscf import gto,scf
import numpy as np 
from numpy.linalg import norm as norm
from scipy.spatial.transform import Rotation as R
import matplotlib.pyplot as plt
from pyscf.grad import rhf as grhf
from pyscf.hessian import rhf as hrhf
import basis_set_exchange as bse
from FDcoeffs import *
from pyscf.geomopt.berny_solver import optimize
ang2bohr=1.8897261246
bohr2ang=.5291772109
%load_ext autoreload
%autoreload 2
from AP_class import alch_deriv,first_deriv_nuc_nuc,second_deriv_nuc_nuc
from AP_symmetry import benz_Symm

In [2]:
benz_atom=""" 
C         6.95191249e-13  2.61173060e+00 -1.67580194e-08 ;
H         2.83052087e-13  4.63941350e+00 -1.15976219e-08 ;
C         2.26182505e+00  1.30586530e+00 -2.37508246e-08 ; 
H         4.01784995e+00  2.31970675e+00 -9.84687205e-09 ; 
C         2.26182505e+00 -1.30586530e+00  8.26065094e-09 ; 
H         4.01784995e+00 -2.31970675e+00  5.83890508e-08 ; 
C        -8.45954456e-13 -2.61173060e+00 -1.55922267e-08 ; 
H         1.49445304e-12 -4.63941350e+00 -3.26774154e-08 ; 
C        -2.26182505e+00 -1.30586530e+00 -8.28266973e-09 ; 
H        -4.01784995e+00 -2.31970675e+00  1.87236306e-09 ; 
C        -2.26182505e+00  1.30586530e+00  1.68546675e-08 ; 
H        -4.01784995e+00  2.31970675e+00  3.31289175e-08 ; 
"""
Benz = gto.M(atom=benz_atom,unit='bohr',basis={"H":"sto-3g","C":"cc-pvtz"},symmetry = 'D6v') #basis={"H":"pc-2",'C':bse.get_basis("pcX-2",fmt="nwchem",elements=[6])})

In [3]:
BenzHF=scf.RHF(Benz)
BenzHF.scf()

converged SCF energy = -230.743864823779


-230.7438648237786

In [4]:
C=BenzHF.mo_coeff
S=BenzHF.get_ovlp()

In [5]:
O=np.diag(BenzHF.mo_occ)

In [6]:
C.T@S@C

array([[ 1.00000000e+00,  6.97616335e-17, -2.95843197e-17, ...,
         4.29344060e-15,  1.09991009e-13,  1.43982049e-15],
       [-2.60494819e-16,  1.00000000e+00,  1.12867479e-15, ...,
        -1.40517806e-13, -3.03576608e-16, -1.86675328e-12],
       [ 6.62684697e-17,  1.47864339e-15,  1.00000000e+00, ...,
        -1.86482774e-17, -5.34641775e-14,  3.42911463e-15],
       ...,
       [ 3.06272111e-15, -1.40284234e-13,  3.89072038e-16, ...,
         1.00000000e+00,  3.08916683e-13, -9.76536289e-14],
       [ 1.10079772e-13,  3.60568903e-16, -5.41120682e-14, ...,
         2.97660497e-13,  1.00000000e+00,  6.71783592e-14],
       [ 1.10623826e-16, -1.86655093e-12,  2.01339033e-15, ...,
        -5.59263465e-14,  3.64941367e-14,  1.00000000e+00]])

In [7]:
from AP_symmetry import rotate_grad,rotate_matrix

In [8]:
from scipy.spatial.transform import Rotation as R

In [9]:
import itertools
%autoreload 2
from AP_class import DeltaV,parse_charge,alchemy_cphf_deriv,make_dP
from AP_class import APDFT_perturbator as AP
from pyscf.symm.basis import _ao_rotation_matrices as aorm

In [146]:
bs=benz_Symm(Benz)

In [195]:
ap=AP(BenzHF,sites=[0,2,4,6],symm=bs)
ap.build_all()

In [200]:
ap.APDFT3([1,0,1,0])

-262.7307913112914

In [201]:
ap.APDFT3([0,1,0,1])

-262.7307913112914

In [202]:
ap.hessian

array([[-2.77779377,  0.28468546,  0.20763277,  0.22598709],
       [ 0.28468546, -2.77779377,  0.28468546,  0.20763277],
       [ 0.20763277,  0.28468546, -2.77779377,  0.28468546],
       [ 0.22598709,  0.20763277,  0.28468546, -2.77779377]])

In [203]:
ap.hessian_nuc_nuc(0,2,4)

array([[0.        , 0.38288788, 0.22106042],
       [0.38288788, 0.        , 0.38288788],
       [0.22106042, 0.38288788, 0.        ]])

In [204]:
dV=DeltaV(Benz,[[0],[1]])

In [205]:
axis=np.asarray([0,0,1])
R.from_rotvec(axis*np.pi/3*(2)).as_dcm(),rm_p

(array([[-0.5      , -0.8660254,  0.       ],
        [ 0.8660254, -0.5      ,  0.       ],
        [ 0.       ,  0.       ,  1.       ]]),
 array([[ 0.5      , -0.8660254,  0.       ],
        [ 0.8660254,  0.5      ,  0.       ],
        [ 0.       ,  0.       ,  1.       ]]))

In [206]:
p_idxs=[i for i,elem in enumerate(Benz.ao_labels()) if  "px" in elem]
d_idxs=[i for i,elem in enumerate(Benz.ao_labels()) if  "dxy" in elem]
f_idxs=[i for i,elem in enumerate(Benz.ao_labels()) if  "fy^3" in elem]

In [207]:
ang=np.pi/3
rm_p=np.asarray([[np.cos(ang),-np.sin(ang),0],\
                 [np.sin(ang),np.cos(ang),0],\
                [0,0,1]])
s_rm,p_rm,d_rm,f_rm=aorm(Benz,rm_p)

In [208]:
rm=np.eye(Benz.nao)
for i in p_idxs:
    rm[i:i+3,i:i+3]=p_rm
for i in d_idxs:
    rm[i:i+5,i:i+5]=d_rm
for i in f_idxs:
    rm[i:i+7,i:i+7]=f_rm

In [209]:
np.allclose(bs.make_RM(2,0),rm.T)

True

In [210]:
np.allclose(ap.dV(0),rotate_matrix(rm.T@ap.dV(2)@rm,Benz,0,ref_site=2),atol=1e-3)

True

In [211]:
dm0=BenzHF.make_rdm1()

In [212]:
np.allclose(dm0,rotate_matrix(rm@dm0@rm.T,Benz,2,ref_site=0),atol=1e-4)

True

In [213]:
ap.mo1(0).shape

(186, 21)

In [214]:
#symm.Dmatrix.get_euler_angles(coords[0],coords[2])

In [215]:
#symm.geom.detect_symm(Benz.atom_and_coords())

In [216]:
np.allclose(ap.e1(0),(C.T@S@rotate_matrix(rm.T@(C[:,:21]@ap.e1(2)@C.T[:21,:])@rm,Benz,0,ref_site=2)@S@C)[:21,:21],atol=1e-3)

True

In [217]:
np.allclose(ap.mo1(0),(C.T@S@rotate_matrix(rm.T@(C@ap.mo1(2)@C.T[:21,:])@rm,Benz,0,ref_site=2)@S@C)[:,:21],atol=1e-3)

True

In [218]:
np.allclose(ap.mo1(2),(C.T@S@rotate_matrix(rm@(C@ap.mo1(0)@C.T[:21,:])@rm.T,Benz,2,ref_site=0)@S@C)[:,:21],atol=1e-3)

True

In [219]:
np.allclose(ap.mo1(2),bs.rotate_mo1e1(ap.mo1(0),ap.e1(0),2,0,C,S)[0],atol=1e-6)

True